In [4]:
import os
import pandas as pd
import logging
import io
import requests
import threading
import queue
from tqdm import tqdm
import time
from bs4 import BeautifulSoup
from lxml import html

In [5]:
headers = {'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
           'sec-ch-ua-mobile':'0',
           'Upgrade-Insecure-Requests': '1',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
                    }

def get_inn_vie_purch(purch,q):   
    url="https://zakupki.gov.ru/epz/order/notice/ea44/view/supplier-results.html?regNumber="+purch 
    r = requests.get(url, headers = headers)
    text = r.content.decode('utf-8')
    soup = BeautifulSoup(text)
    winer_list = soup.find_all('table',{'class': 'blockInfo__table tableBlock'})
    if len(winer_list) > 3:
        soup = BeautifulSoup(str(winer_list[3]))
    elif len(winer_list) == 3:
        soup = BeautifulSoup(str(winer_list[2]))
    else:
        q.put(0)
        return
    contract_link = soup.find('a')
    try:
        contract_url = contract_link.get('href')
    except Exception:
        q.put(0)
        return
    contract_url = 'https://zakupki.gov.ru' + contract_url
    try:
        contract_page = requests.get(contract_url, headers = headers)
    except Exception:
        q.put(0)
        return
    contract_page_text = contract_page.content.decode('utf-8')
    soup = BeautifulSoup(contract_page_text)
    spans = soup.find_all('span')
    ind_cur = -1
    for span in spans:
        if span.text == 'ИНН:': 
            ind_cur = spans.index(span)+1
            break
    if ind_cur == -1:
        q.put(0)
        return 
    else:
        q.put(spans[ind_cur].text)
        return 
tqdm.pandas()

In [6]:
def start_download(purch):
    out = purch
    q = queue.Queue()
    scraping = threading.Thread(target=get_inn_vie_purch, args=(out,q,))
    scraping.start()
    scraping.join()
    while threading.active_count()>15:
        time.sleep(0.01)
    return q.get()

In [7]:
df = pd.read_csv('C:\\Users\\Administrator\\Downloads\\out_file_by_inn_223_44_last.csv', encoding='utf-16', sep='\t')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (11,14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
df.shape

(10946943, 25)

In [9]:
df['purch'] = df['Реестровый номер закупки'].progress_apply(lambda x:x[1:])

100%|██████████| 10946943/10946943 [00:08<00:00, 1343878.99it/s]


In [11]:
df = df.loc[df['Закупки по']=='44-ФЗ']
df = df.loc[df['Этап закупки']=='Определение поставщика завершено']

In [12]:
df["Дата обновления"] = pd.to_datetime(df["Дата обновления"], format="%y.%m.%Y")

In [13]:
df["Год обновления"] = df["Дата обновления"].progress_apply(lambda x: x.year)

100%|██████████| 6802186/6802186 [00:20<00:00, 329575.61it/s] 


In [14]:
df = df[df["Год обновления"].isin([2019, 2020, 2021])]

In [19]:
df["Дата обновления"]

129356   2021-06-01
129357   2021-06-01
129358   2021-06-01
129359   2021-06-01
129360   2021-06-01
            ...    
458515   2020-07-01
458516   2020-07-01
458517   2020-07-01
458518   2020-07-01
458519   2020-07-01
Name: Дата обновления, Length: 60000, dtype: datetime64[ns]

In [15]:
df.shape

(2152814, 27)

In [16]:
start_index = 30000
end_index = 90000
df = df[start_index:end_index]

In [24]:
import pickle

In [18]:
df['winer'] = df['purch'].progress_apply(start_download)

100%|██████████| 60000/60000 [12:14:32<00:00,  1.36it/s]   


In [25]:
with open('purch_winer_dicts\\0_60000.pickle', 'wb') as handle:
    pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
d = pd.Series(df.winer.values, index=df.purch).to_dict()

In [49]:
df.groupby("purch").winer.count()[df.groupby("purch").winer.count() > 1]

purch
0135100005420000062    2
0349200006419000002    2
Name: winer, dtype: int64

In [51]:
df[df.purch.isin(["0135100005420000062", "0349200006419000002"])]

,Закупки по,Реестровый номер закупки,"Способ определения поставщика (подрядчика, исполнителя), подрядной организации (размещения закупки)",Наименование закупки,Предмет электронного аукциона (только для ПП РФ 615),Идентификационный код закупки,Номер лота,Наименование лота,Начальная (максимальная) цена контракта,Валюта,...,Дата размещения,Дата обновления,Этап закупки,Особенности размещения заказа\ Преимущества (44-ФЗ),Дата начала подачи заявок,Дата окончания подачи заявок,Дата проведения электронного аукциона,purch,Год обновления,winer
50715,44-ФЗ,№0349200006419000002,Закрытый аукцион,Поставка лекарственных средств и изделий медиц...,NaN,"192519150190351900100100090013250244, 19251915...",1.0,Поставка лекарственных средств,212967.6,RUB,...,01.03.2019,2020-04-01,Определение поставщика завершено,NaN,01.03.2019,03.04.2019,NaN,0349200006419000002,2020,0
50716,44-ФЗ,№0349200006419000002,Закрытый аукцион,Поставка лекарственных средств и изделий медиц...,NaN,"192519150190351900100100090013250244, 19251915...",2.0,Поставка изделий медицинского назначения,2920437.4,RUB,...,01.03.2019,2020-04-01,Определение поставщика завершено,NaN,01.03.2019,03.04.2019,NaN,0349200006419000002,2020,0
86547,44-ФЗ,№0135100005420000062,Закрытый конкурс с учетом положений ст. 111 За...,Оказание услуг по профессиональному обучению с...,NaN,"201390801348039060100100370018531244, 20139080...",1.0,NaN,1228000.0,RUB,...,01.04.2020,2020-09-01,Определение поставщика завершено,NaN,NaN,07.05.2020,NaN,0135100005420000062,2020,0
86548,44-ФЗ,№0135100005420000062,Закрытый конкурс с учетом положений ст. 111 За...,Оказание услуг по профессиональному обучению с...,NaN,"201390801348039060100100370018531244, 20139080...",2.0,NaN,216000.0,RUB,...,01.04.2020,2020-09-01,Определение поставщика завершено,NaN,NaN,07.05.2020,NaN,0135100005420000062,2020,0


In [90]:
df.to_excel('res_60К.xlsx')

In [82]:
pwd

'C:\\Users\\Administrator'

In [20]:
df.shape

(60000, 28)

In [21]:
df.head(5)

,Закупки по,Реестровый номер закупки,"Способ определения поставщика (подрядчика, исполнителя), подрядной организации (размещения закупки)",Наименование закупки,Предмет электронного аукциона (только для ПП РФ 615),Идентификационный код закупки,Номер лота,Наименование лота,Начальная (максимальная) цена контракта,Валюта,...,Дата размещения,Дата обновления,Этап закупки,Особенности размещения заказа\ Преимущества (44-ФЗ),Дата начала подачи заявок,Дата окончания подачи заявок,Дата проведения электронного аукциона,purch,Год обновления,winer
129356,44-ФЗ,№0369200018921000065,Электронный аукцион,Поставка расходных материалов из пластика для КДЛ,NaN,'212745104017574510100100050050000244',NaN,NaN,427185.00,RUB,...,01.06.2021,2021-06-01,Определение поставщика завершено,"Участникам, заявки или окончательные предложен...",01.06.2021,09.06.2021,NaN,0369200018921000065,2021,2540221551
129357,44-ФЗ,№0190300001221000378,Электронный аукцион,поставка компьютерной техники,NaN,'213890302052489030100100490010000244',NaN,NaN,818533.05,RUB,...,01.06.2021,2021-06-01,Определение поставщика завершено,"Участникам, заявки или окончательные предложен...",01.06.2021,11.06.2021,NaN,0190300001221000378,2021,665800615409
129358,44-ФЗ,№0339100023721000017,Электронный аукцион,Оказание услуг по уборке и обслуживанию зданий...,NaN,'211421702748542170100100060018121244',NaN,NaN,5923433.52,RUB,...,01.06.2021,2021-06-01,Определение поставщика завершено,NaN,01.06.2021,09.06.2021,NaN,0339100023721000017,2021,420506674093
129359,44-ФЗ,№0369200018921000066,Электронный аукцион,"Поставка микроцентрифужных пробирок, объем 0,2мл",NaN,'212745104017574510100100050060000244',NaN,NaN,283200.00,RUB,...,01.06.2021,2021-06-01,Определение поставщика завершено,"Участникам, заявки или окончательные предложен...",01.06.2021,09.06.2021,NaN,0369200018921000066,2021,3328475402
129360,44-ФЗ,№0127200000221002809,Электронный аукцион,Закупка изделий медицинского назначения,NaN,'212320100089532570100100150160000244',NaN,NaN,421825.00,RUB,...,01.06.2021,2021-06-01,Определение поставщика завершено,"Участникам, заявки или окончательные предложен...",01.06.2021,09.06.2021,NaN,0127200000221002809,2021,3257054518
